In [6]:
from glob import glob
import os
import xarray as xr
import tifffile
import numpy as np

files = glob('selected_zarrs/*.zarr')

# output folders
outdirs = {
    "full": "selected_tiffs",
}

for d in outdirs.values():
    os.makedirs(d, exist_ok=True)

for file in files:
    ds = xr.open_zarr(file)
    img = ds["_image"].values  # numpy array

    # save full image
    base = os.path.basename(file).replace(".zarr", ".tiff")
    tifffile.imwrite(os.path.join(outdirs["full"], base), img, photometric="minisblack")

    # spatial dimensions (assuming img shape is (X, Y, channels) or (channels, X, Y))
    if img.ndim == 3:
        _, H, W = img.shape
        channel_first = True

    # save cropped subsets
    for size, outdir in outdirs.items():
        if size == "full":
            continue

        if H < size or W < size:
            print(f"Skipping {base} for size {size}, image too small. {H=}, {W=}, {size=}")
            continue

        minx = (H - size) // 2
        miny = (W - size) // 2
        maxx = minx + size
        maxy = miny + size

        if channel_first:
            img_crop = img[:, minx:maxx, miny:maxy]
        else:
            img_crop = img[minx:maxx, miny:maxy, :]

        outpath = os.path.join(outdir, base.replace(".tiff", f"_{size}.tiff"))
        tifffile.imwrite(outpath, img_crop, photometric="minisblack")

    print(f"Saved {outpath}")